In [15]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import pandas as pd 
import numpy as np
from torch.utils.data import Dataset,DataLoader 


In [16]:
# setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_df = pd.read_csv("data/fashion-mnist_train.csv")
test_data_df = pd.read_csv("data/fashion-mnist_test.csv")
train_data_df


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# model

class FashionCNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1,32, kernel_size=3,padding=1) # 32x28x28
        self.pool = nn.MaxPool2d(2, 2)  # 32x14x14
        self.conv2 = nn.Conv2d(32,64,kernel_size=3,padding=1) # 64x14x14

        self.fc1 = nn.Linear(64*7*7,128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self,x):

        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + Pool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + Pool
        x = x.view(-1, 64 * 7 * 7)            # Flatten to feed to fully connected layers
        x = F.relu(self.fc1(x))               # FC1 + ReLU
        x = self.fc2(x)                       # FC2

        return x
        
        


In [18]:
# custom dataset
class FashionMNISTDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data : pd.DataFrame = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):

        # start from index 1, first one is the label int8 is enough cause 0 to 255
        image = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape(28, 28, 1)
        
        label = self.data.iloc[idx, 0]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


In [19]:
# load the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FashionMNISTDataset(train_data_df,transform=transform)
test_dataset = FashionMNISTDataset(test_data_df,transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
# setup 
model = FashionCNN().to(device)
criterion : nn .CrossEntropyLoss = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)


In [21]:
#training the model

num_epochs = 10 

for epoch in range(num_epochs):

    model.train()
    running_loss = 0.0

    # correct in training
    correct = 0
    total = 0
    
    # retreive a batch of images and labels and feed them to the device
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss  = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()

        # compute the new gradients, and then update the weights
        loss.backward()
        optimizer.step()
        
        #this is the total loss for the entire train set
        running_loss += loss.item()

        #get the max value of the softmax function second item of tuple is the index, the first one is the value
        _, predicted_index = torch.max(outputs.data, 1) 
        total += labels.size(0)
        correct += (predicted_index == labels).sum().item()
    
    # Print statistics
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    
    # Validation mode
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    # this temporarily disables the autograd engine for faster computations
    # no backward() call so 
    with torch.no_grad():

        for images, labels in test_loader:

            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(test_loader)
    val_acc = 100 * val_correct / val_total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

Epoch [1/10], Train Loss: 0.4367, Train Acc: 84.29%, Val Loss: 0.3314, Val Acc: 87.81%
Epoch [2/10], Train Loss: 0.2818, Train Acc: 89.79%, Val Loss: 0.2589, Val Acc: 90.31%
Epoch [3/10], Train Loss: 0.2365, Train Acc: 91.27%, Val Loss: 0.2354, Val Acc: 91.52%
Epoch [4/10], Train Loss: 0.2039, Train Acc: 92.53%, Val Loss: 0.2186, Val Acc: 92.14%
Epoch [5/10], Train Loss: 0.1774, Train Acc: 93.45%, Val Loss: 0.2195, Val Acc: 91.88%
Epoch [6/10], Train Loss: 0.1542, Train Acc: 94.33%, Val Loss: 0.2236, Val Acc: 92.15%
Epoch [7/10], Train Loss: 0.1349, Train Acc: 95.02%, Val Loss: 0.2273, Val Acc: 92.13%
Epoch [8/10], Train Loss: 0.1152, Train Acc: 95.75%, Val Loss: 0.2384, Val Acc: 92.29%
Epoch [9/10], Train Loss: 0.0970, Train Acc: 96.45%, Val Loss: 0.2440, Val Acc: 92.26%
Epoch [10/10], Train Loss: 0.0833, Train Acc: 96.92%, Val Loss: 0.2460, Val Acc: 92.44%


In [22]:
# testing the model with the test set now use eval mode for faster computation
model.eval()

test_loss = 0
correct = 0
total = 0


with torch.no_grad():

    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # orward pass
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Get predictions
        _, predicted = torch.max(outputs.data, 1)
        
        # Update statistics
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct / total

print(f'\nTest Results:')
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')


Test Results:
Test Loss: 0.2460
Test Accuracy: 92.44%
